In [1]:
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np
mysql_setting = {
    'host' : "10.10.10.4",
    'dbname': "spider",
    'user': "root",
    'password': "qwerasdf"
}

def getDatabaseConn():
    engine = create_engine('mysql://{user}:{password}@{host}:3306/{dbname}'.format(**mysql_setting), echo=False)
    conn = engine.connect()
    return conn
conn = getDatabaseConn()

In [2]:
sql_lagou = """
    select * from lagou_recruit_day
"""
lagou_df = pd.read_sql(sql_lagou, conn)

In [3]:
lagou_df = lagou_df[~lagou_df['department'].isnull()]
def salarySplit(line):
    import re
    res = re.match(r'([\d]+)K-([\d]+)K', line)
    if not res:
        res = re.match(r'([\d]+)k-([\d]+)k', line)
    if res:
        salary_low = res[1]
        salary_high = res[2]
        salary_mean = (int(salary_low) + int(salary_high)) / 2
    else:
        res = re.match(r'([\d]+).*', line)
        salary_low = res[1]
        salary_high = res[1]
        salary_mean = res[1]
    return pd.Series([salary_low, salary_high, salary_mean])
tmp_lagou = lagou_df['salary'].apply(salarySplit).rename(columns={0:'salary_low', 1:'salary_high', 2:'salary_mean'})
lagou_df = lagou_df.combine_first(tmp_lagou)

In [24]:
lagou_df[(lagou_df.department.str.contains('百度'))].pivot_table(index=['city'], aggfunc='sum')

,company_id,created_at,id,position_id,publisher_id,salary_high,salary_low,salary_mean
city,,,,,,,,
上海,336367,64933663571,3932593,150506785,270034339,1398.0,761.0,1079.5
东莞,313920,4519529754,45919,10449260,20002452,37.0,20.0,28.5
乌鲁木齐,850305,7547350926,586453,18776766,45716110,29.0,22.0,25.5
北京,7464235,316964878118,18727985,740487777,1113298557,6301.0,3678.0,4989.5
南京,1420545,16572653038,734993,34638819,72565999,117.0,64.0,90.5
南昌,954,4519478798,11250,4190179,1944639,21.0,13.0,17.0
厦门,1575,1506666217,92173,3578036,77189,30.0,15.0,22.5
合肥,69840,3013988751,149885,4793185,11260166,16.0,8.0,12.0
大连,791896,6033052512,198906,12962636,31281636,24.0,15.0,19.5


In [25]:
lagou_df[(lagou_df.department.str.contains('阿里')) & (lagou_df.city == '成都')]

,city,company_id,content,created_at,department,district,education,first_type,id,job_nature,...,position_name,publish_time,publisher_id,salary,salary_high,salary_low,salary_mean,second_type,url,work_year
47036,成都,83357,<div>\n <p>岗位描述:</p>\n<p>1、通过电话、工单等方式为阿...,1506592250,阿里云阿里云事业群-飞天六部-售后支持-GSS招聘,高新区,本科,运营/编辑/客服,62699,全职,...,阿里云-售后技术支持（成都）,2017-09-25 16:53:51,8900401,10k-20k,20.0,10.0,15.0,客服,https://www.lagou.com/jobs/3631505.html,1-3年


In [28]:
from datetime import datetime
def timeFormat(line):
    return pd.Series([datetime.strptime(line, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')])
tencent_df = pd.DataFrame(lagou_df[(lagou_df.department.str.contains('腾讯')) & (lagou_df.city == '成都')].sort_values('publish_time'), columns=['position_name', 'salary_mean', 'department', 'publish_time', 'work_year', 'second_type'])
tencent_df = pd.concat([tencent_df, tencent_df.publish_time.apply(timeFormat)], axis=1).rename(columns={0:'date'})

# 腾讯17年9月-18年1月成都招聘情况

In [32]:
# tencent_df.set_index('date').pivot_table(index='date', columns=['second_type', 'work_year']).fillna('--').rename(columns={'salary_mean':'平均薪资/K'}).to_excel('/Users/monstar/Desktop/tencent.xlsx')
tencent_df.set_index('date').pivot_table(index='date', columns=['second_type', 'work_year']).fillna('--').rename(columns={'salary_mean':'平均薪资/K'})

平均薪资/K                                                          \
second_type 前端开发/移动开发 后端开发       品牌/广告   客服 数据开发            测试     软件开发    运营   
work_year        3-5年 1-3年  3-5年  1-3年 1年以下 1-3年 3-5年     3-5年  不限 1-3年 5-10年   
date                                                                            
2017-09-26         --   --    --    --   --   --   --       --  --   --    --   
2017-09-27         --   --    --     6  4.5   --   --       --  --   --    --   
2017-09-28         --   --    --    --   --   --   27  24.6667  --   27    27   
2017-11-02         --   12    --    --   --   --   --       --  --   --    --   
2017-11-28         --   --    --    --   --   --   --       --  --   --    --   
2017-12-23         --   --    --    --   --   24   --       --  --   --    --   
2017-12-25         --   --  22.5    --   --   --   --       --  --   --    --   
2017-12-29         --   --    --    --   --   --   --       --  20   --    --   
2018-02-05       22.5   --    --    --   --   --   --       --  --   --    --   
2018-02-11         --   --    --    --   --   --   --       --  --   --    --   
2018-03-07         30   --    --    --   --   --   --       --  --   --    --   

                                       
second_type        销售      项目管理        
work_year    不限  1-3年 3-5年 3-5年 5-10年  
date                                   
2017-09-26   --    --   --   15    --  
2017-09-27   --  6.75  7.5   --    --  
2017-09-28   --    --   --   --    27  
2017-11-02   --    --   --   --    --  
2017-11-28   --     9   --   --    --  
2017-12-23   --    --   --   --    --  
2017-12-25   --    --   --   --    --  
2017-12-29   --    --   --   --    --  
2018-02-05   --    --   --   --    --  
2018-02-11    9    --   --   --    --  
2018-03-07   --    --   --   --    --

In [68]:
df_zfb = lagou_df[(lagou_df.company_id.isin([80324,153849])) & (lagou_df.city == '成都')]
% matplotlib inline

In [96]:
from datetime import datetime
import seaborn as sbn
sbn.set()
df_zfb = pd.DataFrame(df_zfb, columns=['publish_time','position_name', 'salary_mean', 'second_type', 'department', 'work_year']).sort_values('publish_time')

tmp_zfb = df_zfb.publish_time.apply(timeFormat)
df_zfb = pd.concat([df_zfb, tmp_zfb], axis=1).rename(columns={0:'date'})
# df_zfb[['date', 'position_name', 'salary_mean', 'second_type', 'department', 'work_year']]

# 蚂蚁金服2017年9月-2018年1月招聘薪资情况

In [112]:
df_zfb[['date','salary_mean', 'second_type', 'department', 'work_year']].set_index('date').pivot_table(index='date', columns=['second_type', 'work_year']).fillna('--').rename(columns={'salary_mean':'平均薪资/K'}).to_excel('/Users/monstar/Desktop/zfb.xlsx')

In [117]:
df_zfb[['date', 'position_name','salary_mean', 'second_type', 'department', 'work_year']].set_index('date')

,position_name,salary_mean,second_type,department,work_year
date,,,,,
2017-09-25,蚂蚁金服-Java架构师-国际事业群,27.0,后端开发,蚂蚁金服集团国际事业群-国际数据招聘,3-5年
2017-09-25,蚂蚁金服-资深/高级算法专家-国际事业群,27.0,数据开发,蚂蚁金服集团国际事业群-国际数据招聘,3-5年
2017-09-26,Java高级开发工程师,23.0,后端开发,蚂蚁金服集团国际事业部-成都研发中心招聘,3-5年
2017-09-28,高级java工程师/架构师,25.0,软件开发,蚂蚁金服集团技术风险部招聘,不限
2017-09-28,高级数据＆算法工程师／专家,26.5,数据开发,蚂蚁金融服务集团(成都)支付宝事业群招聘,1-3年
2017-09-28,java 开发工程师,24.0,后端开发,蚂蚁金融服务集团(成都)支付宝营销技术部招聘,1-3年
2017-09-28,java 研发工程师,22.5,后端开发,蚂蚁金融服务集团(成都)支付宝营销技术部招聘,1-3年
2017-09-28,java 高级工程师,22.5,软件开发,蚂蚁金融服务集团(成都)支付宝营销技术部招聘,1-3年
2017-09-28,java 研发工程师,22.5,后端开发,蚂蚁金融服务集团(成都)支付宝营销技术部招聘,3-5年
